# The Battle of Neighborhoods  

## The best location to open a new cafe in New York

In [1]:
import numpy as np 

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [18]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

In [19]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

##### Download and Explore Dataset

In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [7]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [8]:
neighborhoods_data = newyork_data['features']

In [9]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

##### Transform data into Dataframe

In [10]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [11]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [13]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [22]:
newyork_data=neighborhoods

In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

The dataframe has 5 boroughs and 612 neighborhoods.


##### Map of New York City

In [15]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [20]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

##### FourSquare

In [17]:
CLIENT_ID = 'VSMPSOGUWHRBV3VIU12SNPWPL1PWXX5WGVRLHONRUFEYLIRK'
CLIENT_SECRET = '4LJBFR0POYPU4VJPZ00OUQSRCI0LLCQW4TSS2TWJZQJOUD5K' 
VERSION = '20180605'

##### Explore neighborhoods in New York

In [21]:
# Get the top 100 venues that are in each neighborhood within a radius of 500 meters, extract the category of each venue
#clean the json and structure the results json into a pandas dataframe

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
#Create a new dataframe with the venues
newyork_venues = getNearbyVenues(names=newyork_data['Neighborhood'],latitudes=newyork_data['Latitude'],longitudes=newyork_data['Longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [26]:
newyork_venues.shape

(20230, 7)

In [27]:
newyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [28]:
#check how many venues were returned for each neighborhood
newyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,52,52,52,52,52,52
Annadale,26,26,26,26,26,26
Arden Heights,12,12,12,12,12,12
Arlington,16,16,16,16,16,16
Arrochar,44,44,44,44,44,44
Arverne,42,42,42,42,42,42
Astoria,200,200,200,200,200,200
Astoria Heights,26,26,26,26,26,26
Auburndale,40,40,40,40,40,40


In [29]:
print('There are {} uniques categories.'.format(len(newyork_venues['Venue Category'].unique())))

There are 425 uniques categories.


##### Analyze each Neighborhood

In [30]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

newyork_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nigh

In [31]:
newyork_onehot.shape

(20230, 425)

In [34]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()
newyork_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nigh

In [33]:
newyork_grouped.shape

(301, 425)

In [35]:
#print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in newyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = newyork_grouped[newyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
            venue  freq
0     Pizza Place  0.12
1   Deli / Bodega  0.08
2     Supermarket  0.08
3  Discount Store  0.04
4    Intersection  0.04


----Annadale----
            venue  freq
0     Pizza Place  0.15
1             Pub  0.08
2  Cosmetics Shop  0.08
3          Bakery  0.08
4   Train Station  0.08


----Arden Heights----
           venue  freq
0  Deli / Bodega  0.17
1       Pharmacy  0.17
2    Coffee Shop  0.17
3    Pizza Place  0.17
4         Lawyer  0.17


----Arlington----
                 venue  freq
0          Coffee Shop  0.12
1  American Restaurant  0.12
2         Intersection  0.12
3          Pizza Place  0.12
4        Grocery Store  0.12


----Arrochar----
                venue  freq
0       Deli / Bodega  0.09
1            Bus Stop  0.09
2         Pizza Place  0.09
3          Bagel Shop  0.09
4  Italian Restaurant  0.09


----Arverne----
            venue  freq
0       Surf Spot  0.19
1  Sandwich Place  0.10
2   Metro Station  0.10
3   Burrito Place  

                 venue  freq
0   African Restaurant  0.06
1       Cosmetics Shop  0.04
2    French Restaurant  0.04
3   Chinese Restaurant  0.04
4  American Restaurant  0.04


----Charleston----
            venue  freq
0  Cosmetics Shop  0.07
1   Big Box Store  0.07
2     Coffee Shop  0.07
3      Donut Shop  0.03
4  Massage Studio  0.03


----Chelsea----
                venue  freq
0         Coffee Shop  0.08
1         Art Gallery  0.05
2              Bakery  0.04
3  Seafood Restaurant  0.03
4   French Restaurant  0.03


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.10
1           Cocktail Bar  0.05
2                 Bakery  0.05
3  Vietnamese Restaurant  0.03
4    American Restaurant  0.03


----City Island----
                    venue  freq
0  Thrift / Vintage Store  0.08
1      Seafood Restaurant  0.08
2           Grocery Store  0.08
3           Jewelry Store  0.04
4                     Bar  0.04


----City Line----
                 venue  freq
0     

                venue  freq
0       Deli / Bodega  0.10
1  Chinese Restaurant  0.10
2         Pizza Place  0.10
3       Grocery Store  0.06
4                Bank  0.06


----Fieldston----
                  venue  freq
0           Music Venue  0.33
1      Business Service  0.33
2                 Plaza  0.33
3           Yoga Studio  0.00
4  Pakistani Restaurant  0.00


----Financial District----
          venue  freq
0   Coffee Shop  0.10
1  Cocktail Bar  0.04
2   Pizza Place  0.04
3           Gym  0.04
4           Bar  0.04


----Flatbush----
                  venue  freq
0         Deli / Bodega  0.09
1             Juice Bar  0.09
2           Coffee Shop  0.09
3    Mexican Restaurant  0.09
4  Caribbean Restaurant  0.09


----Flatiron----
                     venue  freq
0     Gym / Fitness Center  0.06
1       Italian Restaurant  0.04
2  New American Restaurant  0.04
3              Yoga Studio  0.03
4      Sporting Goods Shop  0.03


----Flatlands----
                  venue  freq
0    

                venue  freq
0  Mexican Restaurant  0.07
1                Café  0.05
2          Restaurant  0.05
3              Lounge  0.05
4            Wine Bar  0.04


----Jackson Heights----
                       venue  freq
0  Latin American Restaurant  0.11
1        Peruvian Restaurant  0.08
2  South American Restaurant  0.06
3          Mobile Phone Shop  0.05
4                     Bakery  0.05


----Jamaica Center----
                   venue  freq
0         Clothing Store  0.08
1      Mobile Phone Shop  0.06
2  Performing Arts Venue  0.04
3            Pizza Place  0.04
4   Caribbean Restaurant  0.04


----Jamaica Estates----
                  venue  freq
0           Bus Station   0.5
1               Dog Run   0.5
2           Yoga Studio   0.0
3  Pakistani Restaurant   0.0
4             Pet Store   0.0


----Jamaica Hills----
                  venue  freq
0     Indian Restaurant  0.09
1  Fast Food Restaurant  0.09
2            Donut Shop  0.09
3              Pharmacy  0.09
4    

                venue  freq
0         Pizza Place  0.10
1  Spanish Restaurant  0.10
2          Donut Shop  0.10
3          Restaurant  0.05
4        Burger Joint  0.05


----Mount Eden----
                  venue  freq
0              Pharmacy  0.14
1           Pizza Place  0.08
2    Chinese Restaurant  0.06
3  Fast Food Restaurant  0.06
4         Grocery Store  0.06


----Mount Hope----
                venue  freq
0         Pizza Place  0.15
1       Grocery Store  0.15
2       Deli / Bodega  0.08
3  Spanish Restaurant  0.08
4       Metro Station  0.08


----Murray Hill----
                 venue  freq
0    Korean Restaurant  0.15
1          Coffee Shop  0.05
2       Sandwich Place  0.04
3  Japanese Restaurant  0.04
4          Pizza Place  0.03


----Neponsit----
                venue  freq
0               Beach  0.67
1                 Bar  0.17
2           Beach Bar  0.17
3         Yoga Studio  0.00
4  Photography Studio  0.00


----New Brighton----
            venue  freq
0        Bus

                  venue  freq
0  Caribbean Restaurant  0.18
1  Fast Food Restaurant  0.12
2         Deli / Bodega  0.06
3        Breakfast Spot  0.06
4            Donut Shop  0.06


----Richmond Hill----
                       venue  freq
0                Pizza Place  0.08
1  Latin American Restaurant  0.08
2                     Lounge  0.08
3                       Bank  0.08
4              Women's Store  0.04


----Richmond Town----
                venue  freq
0  Italian Restaurant  0.25
1          Bagel Shop  0.25
2                Café  0.25
3                 Spa  0.25
4         Yoga Studio  0.00


----Richmond Valley----
                        venue  freq
0                        Bank  0.15
1        Fast Food Restaurant  0.15
2  Construction & Landscaping  0.08
3                        Food  0.08
4               Train Station  0.08


----Ridgewood----
           venue  freq
0         Bakery  0.09
1       Pharmacy  0.07
2  Grocery Store  0.05
3           Café  0.05
4           Bank 

                        venue  freq
0      Thrift / Vintage Store  0.11
1          Italian Restaurant  0.11
2  Construction & Landscaping  0.11
3                Home Service  0.11
4              Cosmetics Shop  0.11


----Travis----
           venue  freq
0          Hotel  0.21
1            Gym  0.14
2  Bowling Alley  0.14
3           Park  0.07
4  Deli / Bodega  0.07


----Tribeca----
                 venue  freq
0   Italian Restaurant  0.06
1                 Park  0.06
2                  Spa  0.04
3             Wine Bar  0.04
4  American Restaurant  0.04


----Tudor City----
                venue  freq
0                Café  0.06
1  Mexican Restaurant  0.06
2                Park  0.06
3    Sushi Restaurant  0.04
4    Greek Restaurant  0.04


----Turtle Bay----
                venue  freq
0         Coffee Shop  0.05
1    Sushi Restaurant  0.05
2  Italian Restaurant  0.05
3            Wine Bar  0.04
4                Park  0.04


----Unionport----
                       venue  freq
0  L

In [36]:
#Convert it into a dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
#create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']

for ind in np.arange(newyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Supermarket,Department Store,Fried Chicken Joint,Spa,Breakfast Spot,Gas Station,Fast Food Restaurant,Grocery Store
1,Annadale,Pizza Place,Dance Studio,Diner,Park,Bakery,Liquor Store,Train Station,Cosmetics Shop,Pharmacy,Restaurant
2,Arden Heights,Pharmacy,Lawyer,Deli / Bodega,Coffee Shop,Pizza Place,Dry Cleaner,Exhibit,Eye Doctor,Factory,Falafel Restaurant
3,Arlington,Deli / Bodega,American Restaurant,Pizza Place,Construction & Landscaping,Grocery Store,Bus Stop,Coffee Shop,Intersection,Filipino Restaurant,Falafel Restaurant
4,Arrochar,Deli / Bodega,Bus Stop,Bagel Shop,Pizza Place,Italian Restaurant,Nail Salon,Cosmetics Shop,Sandwich Place,Pharmacy,Mediterranean Restaurant


##### CLuster the Neighborhoods using k_means

In [67]:
# set number of clusters
kclusters = 5

newyork_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 0, 0, 3, 0, 0], dtype=int32)

In [68]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

newyork_merged = newyork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

newyork_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0.0,Pharmacy,Ice Cream Shop,Donut Shop,Dessert Shop,Food,Laundromat,Sandwich Place,Gas Station,Farmers Market,Field
1,Bronx,Co-op City,40.874294,-73.829939,2.0,Bus Station,Pizza Place,Fried Chicken Joint,Deli / Bodega,Fast Food Restaurant,Grocery Store,Park,Basketball Court,Bagel Shop,Pharmacy
2,Bronx,Eastchester,40.887556,-73.827806,2.0,Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Cosmetics Shop,Donut Shop,Metro Station,Pizza Place,Platform,Convenience Store
3,Bronx,Fieldston,40.895437,-73.905643,0.0,Music Venue,Business Service,Plaza,Women's Store,Fish Market,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market
4,Bronx,Riverdale,40.890834,-73.912585,2.0,Bus Station,Park,Baseball Field,Home Service,Food Truck,Bank,Gym,Plaza,Medical Supply Store,Exhibit


###### Examine Clusters

In [70]:
#Cluster1
newyork_merged.loc[newyork_merged['Cluster Labels'] == 0, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Ice Cream Shop,Donut Shop,Dessert Shop,Food,Laundromat,Sandwich Place,Gas Station,Farmers Market,Field
3,Fieldston,Music Venue,Business Service,Plaza,Women's Store,Fish Market,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market
6,Marble Hill,Gym,Coffee Shop,Yoga Studio,Pizza Place,Big Box Store,Seafood Restaurant,Miscellaneous Shop,Sandwich Place,Pharmacy,Supplement Shop
9,Williamsbridge,Nightclub,Metro Station,Soup Place,Caribbean Restaurant,Bar,Dance Studio,Fish & Chips Shop,Falafel Restaurant,Design Studio,Farm
10,Baychester,Bank,Donut Shop,Men's Store,Sandwich Place,Electronics Store,Pet Store,Fast Food Restaurant,Pizza Place,Supermarket,Convenience Store
11,Pelham Parkway,Pizza Place,Bus Station,Italian Restaurant,Frozen Yogurt Shop,Smoke Shop,Bakery,Bank,Sushi Restaurant,Chinese Restaurant,Gift Shop
12,City Island,Seafood Restaurant,Thrift / Vintage Store,Grocery Store,Arts & Crafts Store,Park,Harbor / Marina,Boat or Ferry,Baseball Field,Bar,Bank
16,Fordham,Mobile Phone Shop,Fast Food Restaurant,Spanish Restaurant,Bank,Donut Shop,Shoe Store,Pizza Place,Supplement Shop,Bakery,Clothing Store
21,Mott Haven,Donut Shop,Spanish Restaurant,Pizza Place,Grocery Store,Mobile Phone Shop,Bookstore,Bakery,Fish & Chips Shop,Gym,Latin American Restaurant
22,Port Morris,Latin American Restaurant,Storage Facility,Distillery,Food Truck,Furniture / Home Store,Brewery,Spanish Restaurant,Metro Station,Donut Shop,Peruvian Restaurant


In [71]:
#Cluster2
newyork_merged.loc[newyork_merged['Cluster Labels'] == 1, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,Somerville,Park,Women's Store,Event Service,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
498,Somerville,Park,Women's Store,Event Service,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [72]:
#Cluster3
newyork_merged.loc[newyork_merged['Cluster Labels'] == 2, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Bus Station,Pizza Place,Fried Chicken Joint,Deli / Bodega,Fast Food Restaurant,Grocery Store,Park,Basketball Court,Bagel Shop,Pharmacy
2,Eastchester,Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Cosmetics Shop,Donut Shop,Metro Station,Pizza Place,Platform,Convenience Store
4,Riverdale,Bus Station,Park,Baseball Field,Home Service,Food Truck,Bank,Gym,Plaza,Medical Supply Store,Exhibit
18,West Farms,Bus Station,Park,Coffee Shop,Bank,Outdoors & Recreation,Lounge,Scenic Lookout,Donut Shop,Playground,Chinese Restaurant
25,Morrisania,Pizza Place,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Discount Store,Donut Shop,Fish Market,Pharmacy,Seafood Restaurant,Sandwich Place
26,Soundview,Chinese Restaurant,Grocery Store,Fried Chicken Joint,Bus Station,Bus Stop,Discount Store,Burger Joint,Breakfast Spot,Mobile Phone Shop,Video Store
42,Pelham Gardens,Bus Station,Pharmacy,Donut Shop,Spanish Restaurant,Boat or Ferry,Chinese Restaurant,Grocery Store,Bus Line,Bank,Sandwich Place
43,Concourse,Grocery Store,Italian Restaurant,Bus Station,Spanish Restaurant,Pizza Place,Donut Shop,Fried Chicken Joint,Caribbean Restaurant,Fast Food Restaurant,Chinese Restaurant
45,Edenwald,Chinese Restaurant,Supermarket,Bus Station,Grocery Store,Women's Store,Financial or Legal Service,Eye Doctor,Factory,Falafel Restaurant,Farm
73,Starrett City,Bus Station,Cosmetics Shop,Pizza Place,Shopping Mall,Caribbean Restaurant,American Restaurant,Bus Stop,Donut Shop,Pharmacy,Farmers Market


In [73]:
#Cluster4
newyork_merged.loc[newyork_merged['Cluster Labels'] == 3, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Kingsbridge,Pizza Place,Bar,Latin American Restaurant,Bakery,Mexican Restaurant,Fried Chicken Joint,Spanish Restaurant,Chinese Restaurant,Sandwich Place,Pharmacy
7,Woodlawn,Deli / Bodega,Pizza Place,Pub,Bar,Rental Car Location,Train Station,Indian Restaurant,Grocery Store,Food Truck,Food & Drink Shop
8,Norwood,Pizza Place,Park,Bank,Chinese Restaurant,Pharmacy,Burger Joint,Caribbean Restaurant,Mexican Restaurant,Coffee Shop,Bus Station
13,Bedford Park,Pizza Place,Mexican Restaurant,Chinese Restaurant,Diner,Deli / Bodega,Sandwich Place,Bakery,Coffee Shop,Baseball Field,Bus Station
14,University Heights,Pizza Place,Deli / Bodega,Fried Chicken Joint,Donut Shop,Grocery Store,Pharmacy,Sandwich Place,Bank,Bakery,Fast Food Restaurant
15,Morris Heights,Deli / Bodega,Bank,Supermarket,Pharmacy,Spanish Restaurant,Grocery Store,Pizza Place,Women's Store,Filipino Restaurant,Eye Doctor
17,East Tremont,Pizza Place,Shoe Store,Café,Paella Restaurant,Bank,Lounge,Fast Food Restaurant,Outdoors & Recreation,Mobile Phone Shop,Fish & Chips Shop
19,High Bridge,Pharmacy,Chinese Restaurant,Spanish Restaurant,Deli / Bodega,Supermarket,Latin American Restaurant,Pizza Place,Sports Club,Asian Restaurant,Park
20,Melrose,Pizza Place,Pharmacy,Grocery Store,Sandwich Place,Gym / Fitness Center,Discount Store,Platform,Donut Shop,Diner,Martial Arts Dojo
28,Throgs Neck,Italian Restaurant,Asian Restaurant,Deli / Bodega,Mobile Phone Shop,Baseball Field,Bar,Liquor Store,Pizza Place,Coffee Shop,Sports Bar


In [74]:
#Cluster5
newyork_merged.loc[newyork_merged['Cluster Labels'] == 4, newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
193,Brookville,Deli / Bodega,Women's Store,Fish Market,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
202,Grymes Hill,Dog Run,Deli / Bodega,Fishing Store,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
499,Brookville,Deli / Bodega,Women's Store,Fish Market,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
508,Grymes Hill,Dog Run,Deli / Bodega,Fishing Store,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field


###### Clearly, neighborhoods in clusters 1,3 and 4 are busy streets with a lot of famous eateries and restaurants. However, clusters 2 and 5 do not have many famous options. Hence, it is recommended to open a cafe in the neighborhoods of these clusters.

In [85]:
recommended=pd.DataFrame({'Neighborhood':['Brookville','Grymes Hill','Somerville']})
recommended

,Neighborhood
0,Brookville
1,Grymes Hill
2,Somerville


In [95]:
recommended_data=neighborhoods[neighborhoods['Neighborhood'] =='Brookville'] 
recommended_data=recommended_data.append(neighborhoods[neighborhoods['Neighborhood'] =='Somerville'])
recommended_data=recommended_data.append(neighborhoods[neighborhoods['Neighborhood'] =='Grymes Hill'])
recommended_data.drop_duplicates(subset=None, keep='first', inplace=True)
recommended_data

,Borough,Neighborhood,Latitude,Longitude
193,Queens,Brookville,40.660003,-73.751753
192,Queens,Somerville,40.597711,-73.796648
202,Staten Island,Grymes Hill,40.624185,-74.087248


In [96]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(recommended_data['Latitude'], recommended_data['Longitude'], recommended_data['Borough'], recommended_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

##### It is recommended that the cafe could be opened in the above neighbourhoods (Brookville, Somerville or Grymes Hill as shown on the map) as there are no much famous cafes